### Convert generated opt output coordinates into spe input files of ligand and optimized complex

In [1]:
# Workflow
# Extract coordinates within opt file
# Convert into spe input file for M06/def2tzvp/Lanl2dz calculationfrom xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure


In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install git+https://github.com/zotko/xyz2graph.git
# python -m pip install git+https://github.com/zotko/xyz2graph.git

  Cloning https://github.com/zotko/xyz2graph.git to c:\users\george\appdata\local\temp\pip-req-build-cg2fnvx_
  Resolved https://github.com/zotko/xyz2graph.git to commit a1ce9d1b9e08c5ecd69d4916e1455f6940f974d6
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/zotko/xyz2graph.git 'C:\Users\George\AppData\Local\Temp\pip-req-build-cg2fnvx_'


In [3]:
import os
from pathlib import Path
import re #Import RegEx
import os
import math
import numpy as np
import pandas as pd
from pathlib import Path
from plotly.offline import offline
import networkx as nx
from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure

In [4]:
directory = os.getcwd()
directory

'C:\\Users\\George\\Desktop\\Research_UCLA\\CIC\\Computational Work\\Project cluster\\structures\\structures_ligand_id'

In [5]:
# Check if all three files (opt, spe-complex and spe-ligand is present for each ligand)

opt_directory = directory + '\\combined-opt-ligand-id'
spe_complex_directory = directory + '\\combined-spe-ligand-id'
complex_xyz_directory = directory + '\\complex_xyz'
ligand_xyz_directory = directory + '\\ligand_xyz'
directory_list = [opt_directory, spe_complex_directory]
replace_list = ['-opt','-spe']

opt_list = []
spe_complex_list =[]
name_lists = [opt_list,spe_complex_list]

for check_directory,word,name_list in zip(directory_list,replace_list,name_lists):
# Read in .out file
    index_count = 0
#     name_list = []
    for subdir,dirs,files in os.walk(check_directory):                  # Loop over each directory, subdirectory and files
        for file in files:                                      # Loop over each file
            if any([file.endswith('.out')]):                    # If file is a .out file
                filename = os.path.join(subdir, file)       # Return path to file
                name = Path(filename).stem           # Extract filename from the end of path and return as a string
                name = name.replace(word,'')     #Remove -opt in string to get just the name of ligand 'pd-x-x-x'
                name_list.append(name)
                
                
print(len(opt_list))
print(len(spe_complex_list))
missing_in_opt_list = np.unique([x for x in spe_complex_list if x not in opt_list])
missing_in_spe_complex_list = np.unique([x for x in opt_list if x not in spe_complex_list])


print('Missing in opt list: ', missing_in_opt_list)
print('missing in spe complex list: ',missing_in_spe_complex_list)

missing_list = []
missing_list.extend(missing_in_opt_list)
missing_list.extend(missing_in_spe_complex_list)
print(missing_list)

with open("missing_list.txt", "w") as output:
    output.write(str(missing_list))

440
440
Missing in opt list:  []
missing in spe complex list:  []
[]


In [14]:
#Define functions

def get_h_atoms(cart_distances):
    sorted_indices = sorted(range(len(cart_distances)), key=lambda i: cart_distances[i])
    second_smallest_index = sorted_indices[1]
    third_smallest_index = sorted_indices[2]
    fourth_smallest_index = sorted_indices[3]
    fifth_smallest_index = sorted_indices[4] 
    return second_smallest_index, third_smallest_index, fourth_smallest_index, fifth_smallest_index

def get_donor_atoms(cart_distances, element_list):
    # Sort the indices based on cartesian distances
    sorted_indices = sorted(range(len(cart_distances)), key=lambda i: cart_distances[i])
    
    # Filter out indices where the corresponding element is 'H'
    filtered_indices = [i for i in sorted_indices if element_list[i] != 'H']

    second_smallest_index = filtered_indices[1]
    third_smallest_index = filtered_indices[2]
    fourth_smallest_index = filtered_indices[3]
    fifth_smallest_index = filtered_indices[4]
    
    return second_smallest_index, third_smallest_index, fourth_smallest_index, fifth_smallest_index

def get_neighbor_atoms(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda i: numbers[i]) 
    return sorted_indices


def xyz_to_dataframe(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        
    # Initialize lists to store data
    atoms = []
    x_coords = []
    y_coords = []
    z_coords = []
    
    # Parse the lines and extract coordinates
    for line in lines:
        parts = line.split()
        atoms.append(parts[0])
        x_coords.append(float(parts[1]))
        y_coords.append(float(parts[2]))
        z_coords.append(float(parts[3]))

    # Create a pandas DataFrame
    data = {
        'Atom': atoms,
        'X': x_coords,
        'Y': y_coords,
        'Z': z_coords
    }

    xyz_df = pd.DataFrame(data)    
    return(xyz_df)

# def generate_ligand_spe(atoms_to_remove,ligand_name):
    
    



### Obtain XYZ coordinates, obtain metal and  ligand atom labels
#### Ligand atom labels for each bond distance away from the metal center is also obtained. This code only works on -opt files.

### Skip this code to only generate spe .com files form opt .out files

In [15]:
xyz_match = ['X           Y           Z']
nbo_match = ['Natural Population Analysis']
unique_atom_list = []
ligand_atom_pair_list = []
missing_nbo_calc = []

# Intialize list to store atom labels
names = []
pd_atoms = []
methyl_atom_1 = []
methyl_atom_2 = []
hydrogens_methyl_1 = []
hydrogens_methyl_2 = []
ligand_atom_1 = []
ligand_atom_2 = []
atom_distance_lists_2 = []
atom_distance_lists_3 = []
atom_distance_lists_4 = []
atom_distance_lists_2_a = []
atom_distance_lists_3_a = []
atom_distance_lists_4_a = []
atom_distance_lists_2_b = []
atom_distance_lists_3_b = []
atom_distance_lists_4_b = []
shortest_path_atom_label_lists = []

for subdir,dirs,files in os.walk(opt_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('-opt.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem.replace('-opt',"")         # Extract filename from the end of path and return as a string
            print(name)
        
            
            mylines = []
            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # myfile = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines list.


#                 # Find XYZ Coordinates

                for line in mylines:
                    if 'NAtoms=' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])
#                         print(natoms)                
                
                xyz_count = 0
                for line in mylines:
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line
                            xyz_count = xyz_count + 1
                
                line_count = 0
                for line in mylines:
                    line_count = line_count + 1
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line
                            xyz_count = xyz_count - 1
                            if xyz_count > 0:
                                continue
                            elif xyz_count == 0:
                                
                                                                               # For loop for generating the XYZ coordinates
                                count = 0
                                xyz = []
                                while count < natoms:
                                    count = count + 1
                                    xyz.append(mylines[line_count + 1])
                                    line_count = line_count +1


                x_coord = []
                y_coord = []
                z_coord = []
                atom_symbol =[]
                element = {"1":'H', "6":"C", "7": "N", "8":"O", "9":"F", "14":"Si", "15":"P", "16":"S", "17":"Cl", "26":"Fe", "35":"Br", "46":"Pd"}
                
                # Generate XYZ file in .txt form, then find xyz coordinates for metal, atom_a and atom_b
                for line in xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])
                    element_number = int(number_list[1])
                    atom_x = float(number_list[3])
                    atom_x = f"{atom_x:.6f}"
                    atom_y = float(number_list[4])
                    atom_y = f"{atom_y:.6f}"
                    atom_z = float(number_list[5])
                    atom_z = f"{atom_z:.6f}"
                
                    # Make xyz coord into .txt file 
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    unique_atoms = list(set(atom_symbol))
                    name_xyz = name + '-complex_xyz.txt'            
                    
                data = {
                    'Atom': atom_symbol,
                    'X': x_coord,
                    'Y': y_coord,
                    'Z': z_coord
                }

                xyz_df = pd.DataFrame(data)

                # Check if output directory for complex xyz exist, create if it doesn't
                os.makedirs(complex_xyz_directory, exist_ok=True)

                # Combine the directory and filename to create the full file path
                complex_xyz_file_path = os.path.join(complex_xyz_directory, name_xyz)

                
#               xyz_df.to_csv(name_xyz, header=False, index=False, sep = " ")          # Generates .txt file                
                new_row = pd.DataFrame({'Atom':'filler', 'X':'', 'Y':'', 'Z':''}, index=[0])
                xyz_df_txt = pd.concat([new_row,xyz_df.loc[:]]).reset_index(drop=True) 
                new_row_2 = pd.DataFrame({'Atom':natoms, 'X':'', 'Y':'', 'Z':''}, index=[0])
                xyz_df_txt = pd.concat([new_row_2,xyz_df_txt.loc[:]]).reset_index(drop=True)

                # Save complex_xyz as text file in the complex_xyz directory            
                xyz_df_txt.to_csv(complex_xyz_file_path, header=False, index=False, sep = " ")          # Generates .txt file   

                                
                # Identify ligand atoms from xyz_df
                condition = (xyz_df['Atom'] == 'Pd')
                pd_row = xyz_df[condition].iloc[0]
                pd_atom = xyz_df[condition].index[0]
                print('Pd number index: ', pd_atom)


                pd_x = pd_row['X']
                pd_y = pd_row['Y']
                pd_z = pd_row['Z']

                #     print(pd_x,pd_y,pd_z)

                cart_distance_pd_list = []
                element_list = []
                for index, row in xyz_df.iterrows():           #Obtain difference between Pd(x,y,z) coordinates and the rest of the atom's cartesian coordinates

                    x_diff = float(pd_x) - float(row['X'])
                    y_diff = float(pd_y) - float(row['Y'])
                    z_diff = float(pd_z) - float(row['Z'])
                    element = row['Atom']
                    
                    cart_distance_pd = np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)

                    cart_distance_pd_list.append(cart_distance_pd)
                    element_list.append(element)
                print(element_list)

                donor_atom_1, donor_atom_2, donor_atom_3, donor_atom_4 = get_donor_atoms(cart_distance_pd_list,element_list)   #Get atoms coordinated to Pd
                
                # pd_row = xyz_df[condition].iloc[0]    

                donor_atoms = [donor_atom_1, donor_atom_2, donor_atom_3, donor_atom_4]   # List of atom indices that is bonded to Pd

                new_donor_atoms = []

                for donor_atom in donor_atoms:
            #         print('Atom index closest to Pd: ', donor_atom)
                    atom_name = xyz_df.iloc[donor_atom,0]
                    if atom_name == 'C':
                        new_donor_atoms.append(donor_atom)   #Obtain index with carbon atom only

                donor_atom_df = xyz_df.loc[new_donor_atoms]

                #Identifying methyl groups attached on Pd
                methyl_atoms = []
                hydrogen_atoms = []
                hydrogen_indices = []

                for donor_atom in new_donor_atoms:    #iterate through the index of each carbon donor atom

                    c_x = xyz_df.iloc[donor_atom,1]   #obtain x y z coordinates for each carbon donor atom
                    c_y = xyz_df.iloc[donor_atom,2]
                    c_z = xyz_df.iloc[donor_atom,3]

                    cart_distance_c_list = []
                    for index, row in xyz_df.iterrows():
                        x_diff = float(c_x) - float(row['X'])
                        y_diff = float(c_y) - float(row['Y'])
                        z_diff = float(c_z) - float(row['Z'])            

                        cart_distance_c = np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)
                        cart_distance_c_list.append(cart_distance_c)

                    h_atom_1, h_atom_2, h_atom_3, h_atom_4 = get_h_atoms(cart_distance_c_list)

                    h_atoms = [h_atom_1, h_atom_2, h_atom_3, h_atom_4]
            #         print(donor_atom, h_atoms)

                    new_h_atoms = []
                    new_h_indices = []

                    for h_atom in h_atoms:
                        atom_name = xyz_df.iloc[h_atom,0]
                        if atom_name == 'H':
                            h_index = h_atom
                            new_h_indices.append(h_index)
                            h_atom = h_atom + 1
                            new_h_atoms.append(h_atom)

            #         print('Index of donor atom: ', donor_atom, 'Number of Hs: ', len(new_h_atoms)) 

                    if len(new_h_atoms) == 3:
                        methyl_atoms.append(donor_atom)
                        hydrogen_atoms.append(new_h_atoms)
                        hydrogen_indices.append(new_h_indices)


            #     print('Methyl atom indices: ', methyl_atoms)
            #     print('Hydrogen atom indices: ', hydrogen_atoms)

                ligand_atoms_label = []
                new_ligand_atoms_label = []
                ligand_atoms_indicies = []


                ligand_atoms_indicies = [x for x in donor_atoms if x not in methyl_atoms] # Remove atom label to only obtain indices related to ligand atom
                for atom in ligand_atoms_indicies:
                    ligand_atoms_label.append(atom + 1)
                # print('Ligand atom label list: ', ligand_atoms_label)
                ligand_atom_pair_list.append(ligand_atoms_label)
#--------------------------------------------------------------------------------------------------------------------------  

#### Identify ligand atoms based on nbo charge calculated in opt file

                for phrase in nbo_match:                                       # iterate through each phrases
                    nbo_match_count = 0
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line
                            nbo_match_count = nbo_match_count + 1
                            count = 0
                            nbo_xyz = []
                            line_number = mylines.index(line) + 6
                            while count < natoms:
                                count = count + 1
                                nbo_xyz.append(mylines[line_number])
                                line_number = line_number + 1

                            with open('nbo.txt', 'w') as filehandle:                   # Save nbo_xyz  into a txt file
                                for listitem in nbo_xyz:
                                    filehandle.write(listitem)

                # Extract NBO charge for metal, atom A and atom B, atom at distance 2,3,4
                if nbo_match_count == 0:
                    print('No NBO Charge calculation for ', name)
                    missing_nbo_calc.append(name)
                    continue


                elif nbo_match_count > 0:    
                    for line in nbo_xyz:
                        number_list = re.findall('-?\d*\.?\d+',line)
                        atom_number = int(number_list[0])

                        if atom_number == ligand_atoms_label[0]:
                            a_nbo = float(number_list[1])
                            print('Ligand atom label: ', atom_number,
                                  ' Ligand atom A nbo: ', a_nbo)

                        elif atom_number == ligand_atoms_label[1]:
                            b_nbo = float(number_list[1]) 
                            print('Ligand atom label: ', atom_number,
                                  ' Ligand atom B nbo: ', b_nbo)                
                
                
                if a_nbo >= b_nbo:
                    new_ligand_atom_a = ligand_atoms_label[0]
                    new_ligand_atom_b = ligand_atoms_label[1]
                elif a_nbo < b_nbo:
                    new_ligand_atom_a = ligand_atoms_label[1]
                    new_ligand_atom_b = ligand_atoms_label[0]
                
                new_ligand_atoms_label.append(new_ligand_atom_a)
                new_ligand_atoms_label.append(new_ligand_atom_b)
                
                print('Atom_a: ', new_ligand_atom_a, 'Atom_b: ', new_ligand_atom_b)
              
                
#--------------------------------------------------------------------------------------------------------------------------                

                    
                #Obtain xyz coordinates for donor atoms
        
                ligand_atom_1_row = xyz_df.loc[new_ligand_atoms_label[0]-1]
                ligand_atom_2_row = xyz_df.loc[new_ligand_atoms_label[1]-1]
                # print('Ligand number index: ', ligand_atom_1_row)

                ligand_atom_1_x = ligand_atom_1_row['X']
                ligand_atom_1_y = ligand_atom_1_row['Y']
                ligand_atom_1_z = ligand_atom_1_row['Z']
                # print('Atom_a_x: ',ligand_atom_1_x)
                # print('Atom_a_y: ',ligand_atom_1_y)                
                # print('Atom_a_z: ',ligand_atom_1_z)                
                
                ligand_atom_2_x = ligand_atom_2_row['X']
                ligand_atom_2_y = ligand_atom_2_row['Y']
                ligand_atom_2_z = ligand_atom_2_row['Z'] 
                # print('Atom_b_x: ',ligand_atom_2_x)
                # print('Atom_b_y: ',ligand_atom_2_y)                
                # print('Atom_b_z: ',ligand_atom_2_z)   
                
                
                #Find atoms at distance 2 (connected to donor atoms)
                
                atom_distance_2 = []
                atom_distance_2_with_a = []
                atom_distance_2_with_b = []
                
                
                for index, row in xyz_df.iterrows():

                    ligand_1_x_diff = float(ligand_atom_1_x) - float(row['X'])
                    ligand_1_y_diff = float(ligand_atom_1_y) - float(row['Y'])
                    ligand_1_z_diff = float(ligand_atom_1_z) - float(row['Z'])

                    ligand_2_x_diff = float(ligand_atom_2_x) - float(row['X'])
                    ligand_2_y_diff = float(ligand_atom_2_y) - float(row['Y'])
                    ligand_2_z_diff = float(ligand_atom_2_z) - float(row['Z'])


                    cart_distance_ligand_1 = np.sqrt(ligand_1_x_diff**2 + ligand_1_y_diff**2 + ligand_1_z_diff**2)
                    if 0.1 < cart_distance_ligand_1 < 1.93 and index != pd_atom:
                        atom_distance_2.append(index+1)
                        atom_distance_2_with_a.append(index+1)


                    cart_distance_ligand_2 = np.sqrt(ligand_2_x_diff**2 + ligand_2_y_diff**2 + ligand_2_z_diff**2)
                    if 0.1 < cart_distance_ligand_2 < 1.93 and index != pd_atom:
                        atom_distance_2.append(index+1)    
                        atom_distance_2_with_b.append(index+1)

                    # Remove duplicates
                atom_distance_2 = list(set(atom_distance_2))
                # print('Atoms at distance 2: ', atom_distance_2)
                # print('Atoms connected to atom a: ',atom_distance_2_with_a)
                # print('Atoms connected to atom b: ',atom_distance_2_with_b)


                # Find atoms at atom distance 3
                atoms_to_avoid_3 = []
                atoms_to_avoid_3_with_a = []
                atoms_to_avoid_3_with_b = []
                
                atom_distance_3 = []
                atom_distance_3_with_a = []
                atom_distance_3_with_b = []

                atoms_to_avoid_3.extend(atom_distance_2)
                atoms_to_avoid_3.append(pd_atom+1)
                atoms_to_avoid_3.append(new_ligand_atoms_label[0])
                atoms_to_avoid_3.append(new_ligand_atoms_label[1])
                
                atoms_to_avoid_3_with_a.append(pd_atom+1)
                atoms_to_avoid_3_with_a.append(new_ligand_atoms_label[0])
                atoms_to_avoid_3_with_a.append(new_ligand_atoms_label[1])
                atoms_to_avoid_3_with_a.extend(atom_distance_2_with_a)
                
                atoms_to_avoid_3_with_b.append(pd_atom+1)
                atoms_to_avoid_3_with_b.append(new_ligand_atoms_label[0])
                atoms_to_avoid_3_with_b.append(new_ligand_atoms_label[1])
                atoms_to_avoid_3_with_b.extend(atom_distance_2_with_b)
                            
                
                # print('Atoms to avoid at distance 3: ',atoms_to_avoid_3)
                # print('Atoms to avoid at distance 3 with a: ',atoms_to_avoid_3_with_a)
                # print('Atoms to avoid at distance 3 with b: ',atoms_to_avoid_3_with_b)
                
                for atom in atom_distance_2:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_3.append(index+1)     
                
                for atom in atom_distance_2_with_a:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_3_with_a.append(index+1)      
                
                for atom in atom_distance_2_with_b:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_3_with_b.append(index+1)      
                

                #Remove duplicates
                atom_distance_3 = list(set(atom_distance_3))
                atom_distance_3 = [x for x in atom_distance_3 if x not in atoms_to_avoid_3]
                # print('Atoms at distance 3: ', atom_distance_3)

                atom_distance_3_with_a = list(set(atom_distance_3_with_a))
                atom_distance_3_with_a = [x for x in atom_distance_3_with_a if x not in atoms_to_avoid_3_with_a]
                # print('Atoms at distance 3 from donor atom A: ', atom_distance_3_with_a)

                atom_distance_3_with_b = list(set(atom_distance_3_with_b))
                # print('Atom 3, b: ', atom_distance_3_with_b)
                atom_distance_3_with_b = [x for x in atom_distance_3_with_b if x not in atoms_to_avoid_3_with_b]
                # print('Atoms at distance 3 from donor atom B: ', atom_distance_3_with_b)
                                    
                #Find atom at distance 4

                atom_distance_4 = []
                atoms_to_avoid_4 = []
                atom_distance_4_with_a = []
                atoms_to_avoid_4_with_a = []
                atom_distance_4_with_b = []
                atoms_to_avoid_4_with_b = []  
                
                atoms_to_avoid_4 = atoms_to_avoid_3
                atoms_to_avoid_4.extend(atom_distance_3)
                
                atoms_to_avoid_4_with_a = atoms_to_avoid_3_with_a
                atoms_to_avoid_4_with_a.extend(atom_distance_3_with_a)
                
                atoms_to_avoid_4_with_b = atoms_to_avoid_3_with_b
                atoms_to_avoid_4_with_b.extend(atom_distance_3_with_b)                
                
                           
                # print('Atoms to avoid at distance 4: ',atoms_to_avoid_4)
                # print('Atoms to avoid at distance 4 from a: ',atoms_to_avoid_4_with_a)
                # print('Atoms to avoid at distance 4 from b: ',atoms_to_avoid_4_with_b)
                
                for atom in atom_distance_3:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_4.append(index+1)     
                            
                for atom in atom_distance_3_with_a:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_4_with_a.append(index+1)                                 
                            
                for atom in atom_distance_3_with_b:
                    atom_row = xyz_df.loc[atom-1]

                    atom_x = atom_row['X']
                    atom_y = atom_row['Y']
                    atom_z = atom_row['Z']

                    for index, row in xyz_df.iterrows():
                        atom_distance_diff_x = float(atom_x) - float(row['X'])
                        atom_distance_diff_y = float(atom_y) - float(row['Y'])
                        atom_distance_diff_z = float(atom_z) - float(row['Z'])

                        cart_distance_atom = np.sqrt(atom_distance_diff_x**2 + atom_distance_diff_y**2 + atom_distance_diff_z**2)
                        if 0.1 < cart_distance_atom < 1.93:
                            atom_distance_4_with_b.append(index+1)                                             
                            
                
                #Remove duplicates
                atom_distance_4 = list(set(atom_distance_4))
                atom_distance_4 = [x for x in atom_distance_4 if x not in atoms_to_avoid_4]
                # print('Atoms at distance 4: ', atom_distance_4)

                atom_distance_4_with_a = list(set(atom_distance_4_with_a))
                atom_distance_4_with_a = [x for x in atom_distance_4_with_a if x not in atoms_to_avoid_4_with_a]
                # print('Atoms at distance 4 from A: ', atom_distance_4_with_a)
                
                atom_distance_4_with_b = list(set(atom_distance_4_with_b))
                atom_distance_4_with_b = [x for x in atom_distance_4_with_b if x not in atoms_to_avoid_4_with_b]
                # print('Atoms at distance 4 from B: ', atom_distance_4_with_b)
               
                              
                # Add 1 to every index to fit label naming
                names.append(name)
                methyl_atom_1.append(methyl_atoms[0]+1)
                methyl_atom_2.append(methyl_atoms[1]+1)
                hydrogens_methyl_1.append(hydrogen_atoms[0])
                hydrogens_methyl_2.append(hydrogen_atoms[1])
                ligand_atom_1.append(new_ligand_atoms_label[0])
                ligand_atom_2.append(new_ligand_atoms_label[1])
                pd_atoms.append(pd_atom+1)
                atom_distance_lists_2.append(atom_distance_2)
                atom_distance_lists_3.append(atom_distance_3)
                atom_distance_lists_4.append(atom_distance_4)
                atom_distance_lists_2_a.append(atom_distance_2_with_a)
                atom_distance_lists_3_a.append(atom_distance_3_with_a)
                atom_distance_lists_4_a.append(atom_distance_4_with_a)
                atom_distance_lists_2_b.append(atom_distance_2_with_b)                
                atom_distance_lists_3_b.append(atom_distance_3_with_b)                   
                atom_distance_lists_4_b.append(atom_distance_4_with_b)                   

                # Generate ligand_spe file by removing palladium and the two methyl atoms
                atoms_to_remove = methyl_atoms + hydrogen_indices[0] + hydrogen_indices[1]
                atoms_to_remove.append(pd_atom)
                # print('atoms_to_remove',atoms_to_remove)
                ligand_xyz_df = xyz_df.drop(atoms_to_remove,axis =0)
                new_row = pd.DataFrame({'Atom':'filler', 'X':'filler', 'Y':'filler', 'Z':'filler'}, index=[0])
                ligand_xyz_df = pd.concat([new_row,ligand_xyz_df.loc[:]]).reset_index(drop=True) 
                #Add extra filler line to fit format for graph generation
                ligand_xyz_filename = name+'-ligand_xyz.txt'

                # Ensure the output directory exists, create it if it doesn't
                os.makedirs(ligand_xyz_directory, exist_ok=True)

                # Combine the directory and filename to create the full file path
                ligand_xyz_file_path = os.path.join(ligand_xyz_directory, ligand_xyz_filename)

                # Save as text file in the ligand_xyz directory
                ligand_xyz_df.to_csv(ligand_xyz_file_path, header=False,index=False,sep=" ")

                #Use ligand_xyz_df to generate a graph to find the atoms for calculating internal angle
                # Using Graph network
                # Identify nodes with xyz coordinates as ID
                # Use removed PdMe2 structure to construct graph
                #Identify atom A and atom B nodes with their xyz coordinates
                # identify shortest path from A --> 
                # store the xyz atoms for these atoms, convert to atom label in the complex from by matching
                # do this code separately to the csv data generator and add into dataframe
                from plotly.offline import offline
                import networkx as nx
                from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure
                # Create the MolGraph object
                mg = MolGraph()

                # # Read the data from the .xyz file in the directory
                # mg.read_xyz(name+'-ligand_xyz.txt')

                # Read the data from the .xyz file in the directory
                mg.read_xyz(ligand_xyz_file_path)

                # Create the Plotly figure object
                fig = to_plotly_figure(mg)

                # Plot the figure
                # offline.plot(fig)

                # Convert the molecular graph to the NetworkX graph
                G = to_networkx_graph(mg)
                #Set donor atom xyz coordinates for matching
                
                atom_a_x = float(ligand_atom_1_x)
                atom_a_y = float(ligand_atom_1_y)
                atom_a_z = float(ligand_atom_1_z)
                atom_a_xyz = (atom_a_x,atom_a_y,atom_a_z)
                atom_b_x = float(ligand_atom_2_x)
                atom_b_y = float(ligand_atom_2_y)
                atom_b_z = float(ligand_atom_2_z)
                atom_b_xyz = (atom_b_x,atom_b_y,atom_b_z)

                starting_node = []
                ending_node = []

                starting_node_value = atom_a_xyz
                ending_node_value = atom_b_xyz

                # Iterate through nodes and find the one with the desired attribute value
                desired_node = None
                for node in G.nodes():
                    # print(G.nodes[node]['xyz'])
                    if G.nodes[node]['xyz'] == starting_node_value:
                        starting_node = node
                        print('Starting node:',node)
                    if G.nodes[node]['xyz'] == ending_node_value:
                        ending_node = node
                        print('Ending node:',node)
                    
                shortest_path_xyz_list = []
                shortest_path_list = nx.shortest_path(G, source=starting_node, target=ending_node)
                for node in shortest_path_list:
                    print(G.nodes[node]['xyz'], ' ', node)                
                    shortest_path_xyz_list.append(G.nodes[node]['xyz'])
                #Create dictionary to store xyz for each ligand, then use it to match with the xyz in the original opt file
                
                shortest_path_xyz_dict = {}
                atom_order = 0
                for xyz in shortest_path_xyz_list:
                    shortest_path_atom = "Atom%d" %atom_order
                    shortest_path_xyz_dict[shortest_path_atom] = xyz
                    atom_order= atom_order + 1
                # print(shortest_path_xyz_dict)
                
                
                shortest_path_list_atom_label = [pd_atom+1]
                for xyz in shortest_path_xyz_dict:                       #Match with target xyz 
                    target_x = shortest_path_xyz_dict[xyz][0]
                    target_y = shortest_path_xyz_dict[xyz][1]
                    target_z = shortest_path_xyz_dict[xyz][2]
                    
                    # print(target_x)
                    # print(target_y)
                    # print(target_z)
                    
                    for index, row in xyz_df.iterrows():
                        if float(target_x) == float(row['X']) and (float(target_y) == float(row['Y']) and float(target_z) == float(row['Z'])):
                            shortest_path_list_atom_label.append(index+1)
                            break
                            
                
                # print('Atom label shortest path: ', shortest_path_list_atom_label)
                shortest_path_atom_label_lists.append(shortest_path_list_atom_label)
                              

data = {
    'Filename' : names,
    'Pd_atom label' : pd_atoms,
    'Methyl_atom_1 label' : methyl_atom_1,
    'Methyl_atom_2 label' : methyl_atom_2,
    'Hydrogens_methyl_1_label' : hydrogens_methyl_1,
    'Hydrogens_methyl_2_label' : hydrogens_methyl_2,
    'Ligand_atom_1 label' : ligand_atom_1,
    'Ligand_atom_2 label' : ligand_atom_2,
    'Atoms_distance_2 label' : atom_distance_lists_2,
    'Atoms_distance_3 label' : atom_distance_lists_3,
    'Atoms_distance_4 label' : atom_distance_lists_4,
    'Atoms_distance_2_a label': atom_distance_lists_2_a,
    'Atoms_distance_3_a label': atom_distance_lists_3_a,
    'Atoms_distance_4_a label': atom_distance_lists_4_a,
    'Atoms_distance_2_b label': atom_distance_lists_2_b,
    'Atoms_distance_3_b label': atom_distance_lists_3_b,
    'Atoms_distance_4_b label': atom_distance_lists_4_b,
    'Atom label shortest paths' : shortest_path_atom_label_lists
    }


atom_label_df = pd.DataFrame(data)


# Modify Atom_label_df to only include those that are have all three files 

mask = ~atom_label_df['Filename'].isin(missing_list)
atom_label_df = atom_label_df[mask]


csv_filename = 'atoms_label.csv'
atom_label_df.to_csv(csv_filename, index=False)
print(atom_label_df.head())

ligand-001-n-n-1
Pd number index:  1
['C', 'Pd', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'H', 'H', 'C', 'H', 'C', 'H', 'C', 'C', 'C', 'H', 'H', 'N', 'N', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
Ligand atom label:  22  Ligand atom B nbo:  -0.55126
Ligand atom label:  23  Ligand atom A nbo:  -0.48882
Atom_a:  23 Atom_b:  22
Ending node: 12
Starting node: 13
(-1.491218, -0.956693, 0.003991)   13
(-0.668134, -2.116302, 0.349288)   9
(0.678885, -2.114263, -0.344581)   0
(1.496369, -0.949341, -0.00377)   12
ligand-002-p-p-1
Pd number index:  1
['C', 'Pd', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'H', 'H', 'C', 'H', 'H', 'P', 'P', 'C', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H', 'C', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H', 'C', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H', 'C', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
Ligand atom label:  16  Ligand atom A nbo:  1.17518
Ligand atom label:  17  Ligand atom B nbo:  0.95542
Atom_a:  16 Atom_b:  17
Starting no

Ligand atom label list:  [4, 1]
Ligand atom label:  1  Ligand atom B nbo:  0.95457
Ligand atom label:  4  Ligand atom A nbo:  -1.01659
Atom_a:  1 Atom_b:  4
Ligand number index:  Atom            P
X        1.429139
Y       -0.263133
Z       -0.106956
Name: 0, dtype: object
Atom_a_x:  1.429139
Atom_a_y:  -0.263133
Atom_a_z:  -0.106956
Atom_b_x:  -1.445068
Atom_b_y:  -1.002000
Atom_b_z:  1.491718
Atoms at distance 2:  [25, 2, 14, 15]
Atoms connected to atom a:  [2, 15, 25]
Atoms connected to atom b:  [14]
Atoms to avoid at distance 3:  [25, 2, 14, 15, 13, 1, 4]
Atoms to avoid at distance 3 with a:  [13, 1, 4, 2, 15, 25]
Atoms to avoid at distance 3 with b:  [13, 1, 4, 14]
Atoms at distance 3:  [3, 8, 44, 16, 17, 55, 26, 27]
Atoms at distance 3 from donor atom A:  [3, 8, 16, 17, 26, 27]
Atom 3, b:  [3, 4, 44, 55]
Atoms at distance 3 from donor atom B:  [3, 44, 55]
Atoms to avoid at distance 4:  [25, 2, 14, 15, 13, 1, 4, 3, 8, 44, 16, 17, 55, 26, 27]
Atoms to avoid at distance 4 from a:  [

Atoms at distance 3:  [34, 3, 8, 46, 48, 49, 53, 23, 57, 61]
Atoms at distance 3 from donor atom A:  [3, 8, 46, 48, 49, 53, 57, 61]
Atom 3, b:  [34, 3, 4, 23]
Atoms at distance 3 from donor atom B:  [34, 3, 23]
Atoms to avoid at distance 4:  [2, 45, 14, 47, 13, 1, 4, 34, 3, 8, 46, 48, 49, 53, 23, 57, 61]
Atoms to avoid at distance 4 from a:  [13, 1, 4, 2, 45, 47, 3, 8, 46, 48, 49, 53, 57, 61]
Atoms to avoid at distance 4 from b:  [13, 1, 4, 14, 34, 3, 23]
Atoms at distance 4:  [5, 7, 12, 24, 25, 35, 36, 50, 51, 52, 54, 55, 56, 58, 59, 60, 62, 63, 64]
Atoms at distance 4 from A:  [5, 7, 12, 14, 50, 51, 52, 54, 55, 56, 58, 59, 60, 62, 63, 64]
Atoms at distance 4 from B:  [2, 35, 36, 5, 24, 25]
atoms_to_remove [18, 14, 20, 21, 19, 16, 17, 15, 12]
(1.872181, 0.883094, 0.090796)
Starting node: 0
(0.426236, 1.937467, 0.653574)
(2.896502, 0.930558, 1.663654)
(2.683258, 2.091318, -1.10026)
(-0.925601, 1.571769, 0.470222)
(0.691711, 3.186853, 1.23028)
(-1.94591, 2.448676, 0.867005)
(-1.401222, 

Atoms at distance 4:  [26, 38, 39, 40, 68, 70, 14, 15, 48, 49, 50, 25, 58, 59]
Atoms at distance 4 from A:  [4, 68, 38, 39, 40, 70, 48, 49, 50, 58, 59]
Atoms at distance 4 from B:  [70, 14, 15, 55, 25, 26]
atoms_to_remove [8, 4, 9, 10, 11, 6, 7, 5, 2]
(-1.581787, -0.119593, -0.164642)
Starting node: 0
(-1.269921, -1.724269, -1.007349)
(-3.121421, 0.481665, -0.958989)
(-0.275509, 0.915344, -0.952425)
(0.797508, -1.02475, 1.322497)
Ending node: 1
(1.805311, -0.334758, 0.427659)
(2.610642, -1.571975, -0.629985)
(3.077615, 0.506623, 1.403184)
(1.074928, 0.91677, -0.668393)
(2.491985, -2.906651, -0.228807)
(3.286219, -1.257043, -1.81512)
(3.059404, -3.918023, -1.001171)
(1.932427, -3.142256, 0.684555)
(3.851207, -2.272125, -2.583305)
(3.364118, -0.215415, -2.153129)
(3.740037, -3.601985, -2.175316)
(2.960262, -4.963475, -0.686527)
(4.377939, -2.023198, -3.511709)
(4.184086, -4.398988, -2.78335)
(2.633832, 1.502341, 2.282965)
(4.430058, 0.157333, 1.363631)
(3.548227, 2.161272, 3.09882)
(1.56

Atoms at distance 4:  [38, 68, 40, 14, 15, 50, 51, 52, 53, 25, 26, 61, 62, 63]
Atoms at distance 4 from A:  [4, 68, 38, 40, 50, 51, 52, 53, 61, 62, 63]
Atoms at distance 4 from B:  [35, 40, 14, 15, 25, 26]
atoms_to_remove [8, 4, 10, 9, 11, 6, 7, 5, 2]
(-1.235789, 0.597278, 0.613785)
Starting node: 0
(0.155307, -0.388385, 1.331452)
(-1.220625, 2.088144, 1.709419)
(-2.672753, -0.429413, 1.152524)
(0.881067, 0.067553, -1.900217)
Ending node: 1
(1.847026, 0.290925, -0.753406)
(1.992077, 2.038056, -0.29112)
(3.496468, -0.232999, -1.28596)
(1.410281, -0.581597, 0.790429)
(1.836902, 2.97972, -1.316288)
(2.245811, 2.460954, 1.017634)
(1.938362, 4.338075, -1.027749)
(1.618932, 2.640269, -2.336927)
(2.347565, 3.821872, 1.300129)
(2.347836, 1.724762, 1.826718)
(2.193649, 4.757753, 0.278347)
(1.809111, 5.075453, -1.828185)
(2.535645, 4.153017, 2.327903)
(2.266529, 5.828394, 0.5038)
(4.663313, 0.425109, -0.886533)
(3.570426, -1.338005, -2.141839)
(5.903981, -0.039681, -1.318249)
(4.604377, 1.302801

...

Atoms at distance 4:  [35, 10, 12, 45, 17, 19, 21, 54, 23, 28, 30]
Atoms at distance 4 from A:  [35, 4, 10, 12, 45, 17, 19, 21, 54, 23, 28, 30]
Atoms at distance 4 from B:  [2, 10]
atoms_to_remove [36, 40, 38, 37, 39, 43, 41, 42, 35]
(-0.55856, -0.320338, 0.505588)
Starting node: 0
(-2.173369, 0.165825, 1.32726)
(-0.627959, -2.163346, 0.80111)
(0.630484, 0.146878, 1.879932)
(-3.383227, 0.145966, 0.608018)
(-2.234597, 0.537185, 2.677029)
(-3.458388, -0.207204, -1.166543)
(-4.596797, 0.435542, 1.231472)
(-2.682986, 0.939753, -1.756183)
Ending node: 4
(-2.784105, -1.507378, -1.329474)
(-4.879877, -0.164153, -1.526182)
(0.431461, -2.992301, 0.388679)
(-1.691694, -2.745892, 1.503324)
(0.796835, -0.732024, 2.964819)
(1.311254, 1.385145, 1.917243)
(-4.635613, 0.774266, 2.580753)
(-5.504311, 0.390934, 0.619528)
(-3.444719, 0.837234, 3.299768)
(-5.593682, 0.997404, 3.066756)
(-3.449708, 1.119011, 4.360126)
(-1.314319, 0.604974, 3.267022)
(1.55548, -0.401527, 4.081232)
(0.297561, -1.707239, 2.94

Atoms at distance 4:  [6, 7, 16, 20, 25, 27, 34, 35, 45, 46, 66, 68, 69, 73, 77, 81]
Atoms at distance 4 from A:  [65, 67, 34, 6, 7, 35, 45, 46, 16, 20, 25, 27]
Atoms at distance 4 from B:  [66, 68, 69, 5, 73, 77, 81]
atoms_to_remove [35, 39, 38, 37, 36, 42, 40, 41, 32]
(-0.142186, 0.894211, 0.120776)
Starting node: 0
(1.46138, 0.127598, -0.001198)
(0.239229, 2.688663, 0.141957)
(-0.511618, 0.560936, 1.899957)
(1.028065, 0.038471, -2.58784)
Ending node: 1
(1.789145, -0.665136, -1.483564)
(3.594849, -0.464375, -1.821566)
(1.23741, -2.427003, -1.357627)
(2.322456, 0.004414, 1.129067)
(2.223046, -1.111461, 1.968586)
(3.248896, 1.009353, 1.427026)
(3.062504, -1.234503, 3.072634)
(1.447945, -1.860984, 1.760059)
(4.082085, 0.886515, 2.536843)
(3.304448, 1.891244, 0.777575)
(3.996321, -0.237955, 3.357582)
(2.975933, -2.109945, 3.726716)
(4.807741, 1.676825, 2.76176)
(4.654463, -0.333547, 4.228875)
(0.588063, 5.458089, -0.109851)
(1.459149, 4.580788, -0.754644)
(-0.469698, 4.951728, 0.64428)
(

Atoms at distance 4:  [66, 67, 38, 6, 10, 42, 47, 16, 49, 20, 56, 57, 28, 29]
Atoms at distance 4 from A:  [66, 5, 38, 67, 42, 47, 49, 21, 56, 57, 28, 29]
Atoms at distance 4 from B:  [6, 10, 16, 20, 27]
atoms_to_remove [57, 61, 58, 60, 59, 62, 64, 63, 54]
(-0.743027, 0.946106, 0.04906)
Starting node: 0
(0.703357, -0.095116, -0.096742)
(-0.089527, 2.629281, -0.222292)
(-1.050049, 0.827337, 1.859979)
(-0.956585, -1.801696, -1.281398)
Ending node: 1
(0.451525, -1.654412, -0.757243)
(1.696013, -1.81942, -2.05642)
(0.777936, -2.863486, 0.548834)
(2.038388, 0.276223, 0.24422)
(1.262437, -1.5466, -3.360002)
(3.050319, -2.071473, -1.805458)
(2.185186, -1.52041, -4.403649)
(0.198115, -1.346432, -3.544715)
(3.533871, -1.769422, -4.150577)
(1.847026, -1.305629, -5.423682)
(3.966405, -2.048087, -2.854014)
(4.257581, -1.747425, -4.973771)
(5.026758, -2.241878, -2.656226)
(3.399538, -2.271762, -0.783503)
(0.286193, -2.60765, 1.835654)
(0.368919, -3.584358, 2.822946)
(-0.190272, -1.645976, 2.062494)

Atoms at distance 4:  [33, 9, 11, 44, 43, 15, 17, 19, 21, 55, 56, 26, 28]
Atoms at distance 4 from A:  [33, 4, 9, 11, 44, 43, 15, 17, 19, 21, 26, 28]
Atoms at distance 4 from B:  [2, 9, 55, 56]
atoms_to_remove [34, 38, 36, 35, 37, 39, 41, 40, 33]
(-1.837871, -0.024077, 0.273806)
Starting node: 0
(-0.914165, -1.025364, 1.548719)
(-2.405923, 1.388101, 1.309317)
(-3.36866, -1.016367, 0.067208)
(0.473439, -0.895457, 1.748659)
(-1.585097, -1.958062, 2.346246)
(1.371904, 0.298562, 0.756041)
(1.150175, -1.639178, 2.714481)
(1.414284, -0.171849, -0.636645)
Ending node: 4
(0.819928, 1.609646, 1.079541)
(2.962141, 0.212165, 1.294148)
(-3.080525, 2.432108, 0.64241)
(-2.135001, 1.531409, 2.669865)
(-4.634518, -0.605282, 0.484202)
(-3.2549, -2.189163, -0.705997)
(0.450226, -2.552378, 3.49853)
(2.226747, -1.501366, 2.851041)
(-0.918369, -2.713662, 3.308554)
(0.981826, -3.137396, 4.257202)
(-1.479167, -3.430646, 3.918957)
(-2.666511, -2.088149, 2.21351)
(-5.776074, -1.324305, 0.126689)
(-4.732204, 0.

(-0.453569, -0.001281, 0.025399)
Starting node: 0
(0.280081, -1.353772, -1.005911)
(-0.921593, -0.840281, 1.590733)
(-2.103593, 0.197254, -0.783083)
(1.668692, -1.575559, -0.962845)
(-0.490093, -2.13391, -1.877283)
(2.747601, -0.627605, 0.149301)
(2.255078, -2.554863, -1.759459)
(2.689225, 0.772553, -0.390403)
Ending node: 4
(2.087469, -0.773044, 1.459512)
(4.077666, -1.229666, 0.013235)
(-1.23878, -0.01711, 2.676207)
(-0.994024, -2.227244, 1.740577)
(-3.294065, -0.363605, -0.309618)
(-2.143058, 0.98824, -1.939927)
(1.473211, -3.326043, -2.617489)
(3.339564, -2.691768, -1.683251)
(0.098058, -3.115212, -2.675416)
(1.941613, -4.095451, -3.244141)
(-0.527882, -3.716914, -3.346207)
(-1.574857, -1.97574, -1.934228)
(-4.495711, -0.15301, -0.988708)
(-3.286777, -0.976056, 0.601348)
(-4.520151, 0.616713, -2.149301)
(-5.422156, -0.597717, -0.604099)
(-3.33807, 1.187924, -2.624157)
(-5.46501, 0.78146, -2.682079)
(-3.35054, 1.8065, -3.529898)
(-1.214294, 1.461113, -2.290573)
(-1.654843, -0.572951

## Convert all opt .out files into spe .com files from opt_directory to spe_complex_directory

In [ ]:


xyz_match = ['X           Y           Z']
for subdir,dirs,files in os.walk(opt_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('-opt.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem.replace('-opt',"")         # Extract filename from the end of path and return as a string
            print(name)
        
            
            mylines = []
            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # myfile = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines list.


                #initialize charge and multiplicity
                charge = None
                multiplicity = None  
                
                # Find and extract the Charge and Multiplicity values

                for line in mylines:
                    if 'Charge =' in line and 'Multiplicity =' in line:
                        # Use regular expressions to extract numbers
                        charge_multiplicity = re.findall(r'Charge\s*=\s*(-?\d+)\s*Multiplicity\s*=\s*(\d+)', line)
                        charge_str, multiplicity_str = charge_multiplicity[0]
                        charge = int(charge_str)
                        print('Charge: ',charge)
                        multiplicity = int(multiplicity_str)
                        print('Multiplicity: ',multiplicity)
                

#                 # Find XYZ Coordinates

                for line in mylines:
                    if 'NAtoms=' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])
#                         print(natoms)                
                
                xyz_count = 0
                for line in mylines:
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line
                            xyz_count = xyz_count + 1
                
                line_count = 0
                for line in mylines:
                    line_count = line_count + 1
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line
                            xyz_count = xyz_count - 1
                            if xyz_count > 0:
                                continue
                            elif xyz_count == 0:
                                
                                                                               # For loop for generating the XYZ coordinates
                                count = 0
                                xyz = []
                                while count < natoms:
                                    count = count + 1
                                    xyz.append(mylines[line_count + 1])
                                    line_count = line_count +1


                x_coord = []
                y_coord = []
                z_coord = []
                atom_symbol =[]
                element = {"1":'H', "6":"C", "7": "N", "8":"O", "9":"F", "14":"Si", "15":"P", "16":"S", "17":"Cl", "26":"Fe", "35":"Br", "46":"Pd"}
                
                # Generate XYZ file in .txt form, then find xyz coordinates for metal, atom_a and atom_b
                for line in xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    # print(number_list)
                    # print(number_list[0])
                    atom_number = int(number_list[0])
                    element_number = int(number_list[1])
                    atom_x = float(number_list[3])
                    atom_x = f"{atom_x:.6f}"
                    atom_y = float(number_list[4])
                    atom_y = f"{atom_y:.6f}"
                    atom_z = float(number_list[5])
                    atom_z = f"{atom_z:.6f}"
                
                    # Make xyz coord into .txt file 
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    unique_atoms = list(set(atom_symbol))
                    name_xyz = name + '-complex_xyz.txt'          

#                     # Make xyz coord into .txt file 
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    unique_atoms = list(set(atom_symbol))
                    name_xyz = name + '-complex_xyz.txt'  
                    name_spe_xyz = name + '-complex_spe_xyz.txt'
                    
                # data = {
                #     'Atom': atom_symbol,
                #     'X': x_coord,
                #     'Y': y_coord,
                #     'Z': z_coord
                # }
                
                # xyz_df = pd.DataFrame(data)
                # xyz_df.to_csv(name_xyz, header=False, index=False, sep = " ")          # Generates .txt file 


                spe_data = {
                    'Name': name,
                    'Charge': charge,
                    'Multiplicity': multiplicity,
                    'Atom': atom_symbol,
                    'X': x_coord,
                    'Y': y_coord,
                    'Z': z_coord
                }
                xyz_spe_df = pd.DataFrame(spe_data).drop_duplicates()
                xyz_spe_df.to_csv(name_spe_xyz, header=False, index=False, sep = " ")          # Generates .txt file
                print(xyz_spe_df)

                os.makedirs(spe_complex_directory, exist_ok=True)


                # Extract unique elements from 'Atom' column, excluding 'Pd'
                elements = xyz_spe_df[xyz_spe_df['Atom'] != 'Pd']['Atom'].unique()
                
                # Convert the list of elements to a string formatted for output
                elements_string = " ".join(sorted(elements)) + " 0"
                                
                # Format the file content
                output_lines = []
                output_lines.append(f"%mem=16000MB\n")
                output_lines.append(f"%nprocshared=16\n")
                output_lines.append(f"%chk={name}-spe.chk\n")
                output_lines.append(f"#p m06/genecp pop=nbo\n\n")
                output_lines.append(f"{name}\n\n")
                output_lines.append(f"{charge} {multiplicity}\n")
                #Add the atom coordinates
                for index, row in xyz_spe_df.iterrows():
                    output_lines.append(f"{row['Atom']} {row['X']} {row['Y']} {row['Z']}\n")
                   
                # Add the additional lines with the dynamic elements_string
                output_lines.append("\n")
                output_lines.append(f"{elements_string}\n")
                output_lines.append("def2tzvp\n")
                output_lines.append("****\n")
                output_lines.append("Pd 0\n")
                output_lines.append("LANL2DZ\n")
                output_lines.append("****\n\n")
                output_lines.append("Pd 0\n")
                output_lines.append("LANL2DZ\n")
                output_lines.append("\n\n")
                # Specify the output path and save the file
                filename = f"{name}-spe.com"  # Replace with the desired filename or use 'name' variable
                output_path = os.path.join(spe_complex_directory, filename)
                
                # Save the output as a .txt file
                with open(output_path, 'w') as file:
                    file.writelines(output_lines)
                
                print(f"File saved to {output_path}")



In [ ]:
xyz_spe_df

### Swap NBO atoms

In [ ]:
atom_label_df